# 1. Introduction

The purpose of this massive Granger Causality Tests is to identify any historical value of the Fred factors to the milk price in different country / regions.  While the other correlation / regression analysis of this study is focusing on matching the values of the same time point, Granger Causality Test is focusing on the time-lagged version of the Fred factors on predicting the future values of milk price.

In [1]:
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests as gct

# 2. Import and Processing Data

In [2]:
EU_Milk = pd.read_pickle('data/eu.pickle')
US_Milk = pd.read_pickle('data/us.pickle')
China_Milk = pd.read_pickle('data/cn.pickle')
Fred = pd.read_pickle('data/fred.pickle')

# 3. Define Functions to Facilitate Batch Granger Causality Test

In [3]:
# This function returns the time logs, from 1 month to 1 year, on whether the 2nd time series Granger-cause the 1st one under the 0.05 p-value.
def gct_lags(s1, s2, lag):
    result = gct(s1.to_frame().join(s2).dropna(), maxlag=lag, verbose=False)
    # extract the result of just the F-test and whether the null hypothesis of "no Granger-cause" is rejected for all 12 time lags
    rejected = [result[l][0]['ssr_ftest'][1] < 0.05 for l in range(1, lag+1)]
    # return a tuple of time lags with null hypothesis rejected
    return tuple(i+1 for i, v in enumerate(rejected) if v)

In [4]:
# Thie function produce the matrix of Granger-casual time lags between 2 dataframes of time series
def gct_lags_matrix(df1, df2, lag):
    matrix = pd.DataFrame([[gct_lags(df1[a], df2[b], lag) for a in df1] for b in df2], index=df2.columns, columns=df1.columns).astype(str)
    # to improve the displace of results for "All" or "None" cases
    return matrix.replace(to_replace={str(tuple(l for l in range(1, lag+1))):'All', '()':'None'})

In [5]:
def get_latex(df,idx_start,idx_end):
    string = df.iloc[:,idx_start:idx_end].to_latex()
    return string.replace('  ','').replace('\\begin{tabular}{llllll}','').replace('\\begin{tabular}{lllllll}','').replace('\\end{tabular}','').replace('\\\\','\\').replace('\\\\\\\\\\\\','\\').replace('\n','').replace('\\toprule','').replace('\\midrule','').replace('\\bottomrule','')


# 4. Results

In [6]:
gct_lags_matrix(EU_Milk, Fred, 6)

,Germany,France,Netherlands,Italy,Poland,Ireland,Spain,Denmark,Belgium,Austria,EU(without UK)
MCOILBRENTEU,"(2, 3, 4, 5, 6)",All,All,All,None,All,"(2, 3, 4)",All,"(2, 3, 4, 5, 6)","(2, 3, 4, 5)",None
MCOILWTICO,"(2, 3, 4, 5, 6)",All,All,All,None,All,"(2, 3, 4, 5)",All,"(2, 3, 4, 5, 6)","(2, 3, 4, 5, 6)",None
MHHNGSP,None,None,None,"(2, 5)",None,None,None,None,None,None,None
PBARLUSDM,All,All,All,All,None,All,All,All,All,All,"(1, 2, 3)"
PMAIZMTUSDM,"(2, 3, 4, 5, 6)","(2, 3, 4, 5)","(1, 2, 3, 4, 5)",All,None,"(2, 3, 4, 5, 6)",None,All,"(2, 3, 4, 5, 6)","(3, 4, 5)","(2, 3)"
PNGASEUUSDM,None,None,"(3,)",None,None,None,None,None,None,None,None
PSOYBUSDM,"(2, 3, 4, 5, 6)",All,All,"(1, 2, 3, 4, 6)",None,All,None,All,"(2, 3, 4, 5, 6)","(2, 3, 4, 5, 6)",None
PWHEAMTUSDM,All,All,All,All,"(2, 3, 6)",All,"(2, 3, 4, 5, 6)",All,"(2, 3, 4, 5, 6)",All,None
IPG32411S,None,"(1, 2, 3, 4, 5)",None,"(1, 2, 3, 4, 5)",None,None,"(1, 2)",None,None,"(3, 4)",None
A33DNO,"(2, 3, 4, 5, 6)",All,"(3, 4, 5, 6)",All,None,"(2, 3, 4, 5, 6)","(2, 3, 4, 5, 6)","(2, 3, 4, 5, 6)","(2, 5)","(2, 3, 4, 5, 6)","(1, 6)"


In [7]:
granger_EU_9205 = gct_lags_matrix(EU_Milk.drop(columns='EU(without UK)').loc['1992-01-01':'2005-12-31'], Fred.loc['1992-01-01':'2005-12-31'], 10)
granger_EU_9205

,Germany,France,Netherlands,Italy,Poland,Ireland,Spain,Denmark,Belgium,Austria
MCOILBRENTEU,None,"(5, 6, 10)","(3,)","(10,)",None,None,None,None,None,"(10,)"
MCOILWTICO,None,"(5, 6)","(3,)","(10,)",None,None,None,None,None,None
MHHNGSP,None,None,None,None,"(7, 8, 9)",None,None,None,None,None
PBARLUSDM,"(3, 4, 5, 6)",None,None,"(1, 8)",None,"(3, 4, 5, 6, 7, 8, 9, 10)",None,None,None,"(3, 4, 5, 6)"
PMAIZMTUSDM,None,"(2,)",None,"(1, 2, 3, 4, 7, 8)","(4, 5, 6, 7, 8, 9, 10)","(2, 3)",None,None,"(3,)",None
PNGASEUUSDM,None,None,None,None,None,None,None,None,None,None
PSOYBUSDM,"(2,)","(2, 3, 4, 5, 6)",None,"(8,)","(5, 7, 9, 10)","(2, 3, 4, 5, 6)",None,"(2, 3, 4, 5, 6)","(2, 3, 4, 7, 9)",None
PWHEAMTUSDM,None,"(2, 3)",None,"(1, 2, 3, 4, 5, 6, 7, 8, 9)",None,None,None,"(4,)","(2, 3, 4, 5)",None
IPG32411S,None,None,None,None,None,None,None,None,None,None
A33DNO,None,None,None,None,None,None,None,None,None,None


In [8]:
granger_EU_0622 = gct_lags_matrix(EU_Milk.loc['2006-01-01':], Fred.loc['2006-01-01':], 12)
granger_EU_0622

,Germany,France,Netherlands,Italy,Poland,Ireland,Spain,Denmark,Belgium,Austria,EU(without UK)
MCOILBRENTEU,"(10, 11, 12)","(5, 7, 8, 10, 11)",All,"(1, 2, 3, 4)",None,"(2, 5)",None,"(1, 2, 3, 4, 5, 6, 7, 8, 9)","(4,)",None,None
MCOILWTICO,"(12,)","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)",All,"(1, 2, 3, 4, 5, 6, 7, 8)","(4, 5, 6)","(2, 3, 5, 6)",None,"(1, 2, 3, 4, 5, 6, 7, 8, 9)","(4, 5)",None,None
MHHNGSP,"(2, 3, 4)","(3, 5, 6, 9, 10, 11, 12)","(11,)","(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)","(2, 3, 4)",None,"(7, 8)",None,"(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)",None,None
PBARLUSDM,"(1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)","(1, 2)","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12)","(1, 2, 3, 8, 9, 10, 11, 12)","(9,)","(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)","(3, 6, 8, 9)",All,"(3, 4, 5, 6, 7, 8, 9, 10, 11, 12)","(3, 6, 8, 9, 10, 11, 12)","(1, 2, 3)"
PMAIZMTUSDM,None,None,None,None,"(8, 9)",None,None,None,"(7, 8, 9, 10, 11)","(7, 8)","(2, 3)"
PNGASEUUSDM,"(1,)",None,None,None,None,None,"(1,)","(1,)",None,"(1,)",None
PSOYBUSDM,"(6, 7, 8, 9, 10, 11)","(7, 8, 9, 10, 11, 12)","(6, 7, 8, 9)",None,None,"(6, 7, 8, 9, 10)","(7, 8)","(2, 3)","(7, 8)",None,None
PWHEAMTUSDM,"(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)",All,"(1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12)",All,"(2, 3, 4, 6, 7, 8, 9, 10, 11, 12)","(2, 3, 6, 7, 8, 9, 10, 12)","(9, 10, 11, 12)",All,"(2, 3, 10, 11, 12)",All,None
IPG32411S,None,None,None,None,None,None,None,None,None,None,None
A33DNO,"(5, 6, 7, 8, 9, 10, 11)","(1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12)",None,"(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)",None,"(5, 6, 7, 8, 9, 10, 11)","(2, 5, 6, 7, 8, 9, 10, 11, 12)","(2, 5, 6, 7, 8, 9, 10, 12)",None,"(2, 3, 5, 6, 7, 8, 9)","(1, 6, 8)"


In [9]:
granger_EU_0622.style.to_latex()

'\\begin{tabular}{llllllllllll}\n & Germany & France & Netherlands & Italy & Poland & Ireland & Spain & Denmark & Belgium & Austria & EU(without UK) \\\\\nMCOILBRENTEU & (10, 11, 12) & (5, 7, 8, 10, 11) & All & (1, 2, 3, 4) & None & (2, 5) & None & (1, 2, 3, 4, 5, 6, 7, 8, 9) & (4,) & None & None \\\\\nMCOILWTICO & (12,) & (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11) & All & (1, 2, 3, 4, 5, 6, 7, 8) & (4, 5, 6) & (2, 3, 5, 6) & None & (1, 2, 3, 4, 5, 6, 7, 8, 9) & (4, 5) & None & None \\\\\nMHHNGSP & (2, 3, 4) & (3, 5, 6, 9, 10, 11, 12) & (11,) & (2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12) & (2, 3, 4) & None & (7, 8) & None & (2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12) & None & None \\\\\nPBARLUSDM & (1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12) & (1, 2) & (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12) & (1, 2, 3, 8, 9, 10, 11, 12) & (9,) & (2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12) & (3, 6, 8, 9) & All & (3, 4, 5, 6, 7, 8, 9, 10, 11, 12) & (3, 6, 8, 9, 10, 11, 12) & (1, 2, 3) \\\\\nPMAIZMTUSDM & None & None & None & None & (8, 9) & 

In [10]:
get_latex(granger_EU_0622,0,5)

C:\Users\Vincent\AppData\Local\Temp\ipykernel_2696\685246038.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  string = df.iloc[:,idx_start:idx_end].to_latex()


'{} & Germany &France & Netherlands & Italy & Poland \\MCOILBRENTEU &(10, 11, 12) & (5, 7, 8, 10, 11) & All &(1, 2, 3, 4) & None \\MCOILWTICO & (12,) & (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11) & All &(1, 2, 3, 4, 5, 6, 7, 8) &(4, 5, 6) \\MHHNGSP& (2, 3, 4) &(3, 5, 6, 9, 10, 11, 12) & (11,) &(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12) &(2, 3, 4) \\PBARLUSDM&(1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12) &(1, 2) & (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12) & (1, 2, 3, 8, 9, 10, 11, 12) & (9,) \\PMAIZMTUSDM&None &None &None &None & (8, 9) \\PNGASEUUSDM&(1,) &None &None &None & None \\PSOYBUSDM&(6, 7, 8, 9, 10, 11) & (7, 8, 9, 10, 11, 12) &(6, 7, 8, 9) &None & None \\PWHEAMTUSDM&(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12) & All &(1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12) & All &(2, 3, 4, 6, 7, 8, 9, 10, 11, 12) \\IPG32411S&None &None &None &None & None \\A33DNO & (5, 6, 7, 8, 9, 10, 11) &(1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12) &None &(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12) & None \\PCEC96 &None &None &None &None & None \\'

In [11]:
gct_lags_matrix(US_Milk, Fred, 12)

State,California,Wisconsin,Idaho,Texas,New York,Michigan,Minnesota,Pennsylvania,New Mexico,Washington,US Total
MCOILBRENTEU,All,All,"(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12)",All,All,All,All,All,All,All,All
MCOILWTICO,All,All,"(1, 2, 3, 4, 5, 6, 7, 8)",All,All,All,All,All,All,All,All
MHHNGSP,None,None,None,None,None,None,None,None,None,None,None
PBARLUSDM,"(1, 2, 3, 4, 5, 6, 7, 8, 11, 12)","(1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12)","(1, 2, 6, 7, 8, 9, 10, 11, 12)","(1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12)",All,All,"(1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12)",All,"(1, 2, 3, 4, 5, 6, 7, 11, 12)","(1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12)","(1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12)"
PMAIZMTUSDM,"(1, 2, 3, 4, 5, 6, 11, 12)","(1, 2, 3, 4, 5, 6, 7, 11, 12)","(1, 2, 3, 4, 5, 6, 7, 11, 12)",All,All,All,"(1, 2, 3, 4, 5, 6, 11, 12)",All,All,All,"(1, 2, 3, 4, 5, 6, 7, 11, 12)"
PNGASEUUSDM,None,"(2, 12)","(12,)","(2, 5, 6, 7, 10, 11, 12)","(2, 6, 7, 8, 9, 10, 11, 12)","(2, 3, 5, 6, 9, 10, 11, 12)","(2, 6, 7, 11, 12)","(2, 6, 9, 10, 11, 12)","(2, 5, 6, 10, 12)","(6, 10, 12)","(2, 11, 12)"
PSOYBUSDM,All,All,All,All,All,All,All,All,All,All,All
PWHEAMTUSDM,"(1, 2, 3)","(1, 2, 3, 4, 5, 7, 8)","(1, 2, 4)","(1, 2, 3)","(1, 2, 3, 4, 5, 10, 11)","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)","(1, 2, 4, 7)","(1, 2, 3, 4, 5)","(1, 2, 3, 5, 9, 10, 11)","(1, 2, 3, 5, 7, 9, 10, 11, 12)","(1, 2, 3, 4)"
IPG32411S,"(3, 4, 5, 6, 7, 8, 9, 10, 11, 12)","(4, 5, 6, 7, 8, 9, 10, 11, 12)","(3, 4, 5, 6, 7, 8, 9, 10, 11, 12)","(3, 4, 5, 6, 7, 8, 9, 10, 11, 12)","(2, 3, 4, 5, 6, 7, 8, 9)","(4, 5, 6, 7)","(3, 4, 5, 6, 7, 8, 9, 10, 11, 12)","(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)","(3, 4, 5, 6, 7, 8, 9, 10, 11, 12)","(3, 4, 5, 6, 7, 8, 9, 10, 11, 12)","(3, 4, 5, 6, 7, 8, 9, 10, 11, 12)"
A33DNO,"(2, 3, 4, 5, 6)",All,"(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)",All,"(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)",All,All,All,"(2, 3, 8, 9)",All


In [12]:
gct_lags_matrix(China_Milk.to_frame(), Fred, 12)

,China
MCOILBRENTEU,"(3, 4, 5)"
MCOILWTICO,None
MHHNGSP,None
PBARLUSDM,"(1,)"
PMAIZMTUSDM,None
PNGASEUUSDM,None
PSOYBUSDM,None
PWHEAMTUSDM,None
IPG32411S,None
A33DNO,None


# 5. Observations and Insights

There are lots of interesting insights from the above Granger-causuality time lags matrices:
1. Milk prices for Poland, EU overall and China are not Granger-causal for almost all Fred factors except for some few time lags.  That might simply because of few availability of data points that affect the test.
1. Milk prices for all US states are Granger-casual by both oil prices for all time lags except Idaho is just not sensitive for a few time lags.
1. Milk prices for most EU countries are Granger-casual by most, if not all time-lags of both oil prices, except for Poland and overall EU.
1. MHHNGSP (Henry Hub Gas price) is not useful for all US states and EU countries except for France and Italy for a few time lags.
1. PBARLUSDM (Barley price) is useful for all US states and EU countries for most (if not all) time lags, except Poland and overall EU.
1. PMAIZMTUSDM (Corn price) is useful for all US states and EU countries for mixed time lags.
1. PNGASEUUSDM (Natural gas price) is useful for US States for some time lags but not useful for most EU countries.
1. PWHEAMTUSDM (Wheat price) is very useful for all EU countries for all time lags and useful for US states for some time lags.
1. While IPG32411S (Petroleum Refineries) is very useful for all US states for most time lags, it's not useful for most EU countries except France and Italy.
1. A33DNO (Machineary Manufacturing) is useful for US states and EU countries for some or most time lags.
1. PCEC96 (personal consumption expenditures) is useful to predict milk prices for all states for most (if not all) time lags, but not useful at all for EU countries.